In [5]:
import requests
from requests import Response
from io import StringIO
from csv import DictReader
def get_aqi_data():
    try:
        aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
        response:Response=requests.get(aqi_url)
        response.raise_for_status()
    except Exception as e:
        print(f'HTTP error occurred:{e}')
        return None
    else:
        if response.status_code != 200:
            print('Status code error.')
            return None
    if response:
        rawString=response.text
    file:StringIO=StringIO(rawString)
    read:DictReader=DictReader(file)
    data:list[dict]=list(read)
    return data
aqi_data=get_aqi_data()

In [20]:
from pydantic import RootModel,BaseModel,Field,field_validator
class Site(BaseModel):
    county:str
    sitename:str
    aqi:float
    creat_time:str=Field(alias='datacreationdate')
    particulate_matter:float=Field(alias='pm2.5')
    @field_validator('particulate_matter',mode='before')
    @classmethod
    def empty_string_pm25(cls,value):
        if value=='':
            return 0.0
        else:
            return value
class Sites(RootModel):
    root:list[Site]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]
    
sites:Sites=Sites.model_validate(aqi_data)
for item in sites:
    print(item)

county='屏東縣' sitename='屏東(枋山)' aqi=23.0 creat_time='2024-04-02 23:00' particulate_matter=8.0
county='臺中市' sitename='大甲（日南國小）' aqi=40.0 creat_time='2024-04-02 23:00' particulate_matter=9.0
county='新北市' sitename='新北(樹林)' aqi=77.0 creat_time='2024-04-02 23:00' particulate_matter=24.0
county='屏東縣' sitename='屏東(琉球)' aqi=27.0 creat_time='2024-04-02 23:00' particulate_matter=8.0
county='臺南市' sitename='臺南(麻豆)' aqi=37.0 creat_time='2024-04-02 23:00' particulate_matter=11.0
county='高雄市' sitename='高雄(湖內)' aqi=25.0 creat_time='2024-04-02 23:00' particulate_matter=7.0
county='彰化縣' sitename='彰化(員林)' aqi=55.0 creat_time='2024-04-02 23:00' particulate_matter=12.0
county='彰化縣' sitename='大城' aqi=28.0 creat_time='2024-04-02 23:00' particulate_matter=7.0
county='新北市' sitename='富貴角' aqi=57.0 creat_time='2024-04-02 23:00' particulate_matter=17.0
county='雲林縣' sitename='麥寮' aqi=28.0 creat_time='2024-04-02 23:00' particulate_matter=7.0
county='臺東縣' sitename='關山' aqi=18.0 creat_time='2024-04-02 23:00' particula